Goal: pick 4 USA men's Gymnasts. Gymnasts cannot compete on the same appartaus more than once. And each apparatus needs to be competed on 3 times each, then get the largest sum of the top 3 out of 4 scores on each apparats

explore all possible combinations of athletes and apparatuses, aiming to find the combination that maximizes the total score. and return the 4 athletes

# Tabu Search

### IAA

Selected Athletes:
['khoi young' 'asher hong' 'fred richard' 'yul moldauer']
 Score:
260.57900000000006

### Apparatus
Selected Athletes:
['frederick nathaniel richard' 'paul juda' 'khoi young' 'brody malone']
 Score:
253.19598690476192

---

# Hyper-Heuristics:

### IAA
Total Score: ['joshua karnes' 'ian lasic-ellis' 'asher hong' 'yul moldauer']
 Score: 254.94

### Apparatus
Total Score: ['khoi young' 'paul juda' 'brody malone' 'fuzzy benas']
 Score: 250.58868690476191

---


#  Variable Neighborhood Search (VNS):

## IAA

Selected Athletes: ['fuzzy benas', 'yul moldauer', 'asher hong', 'fred richard']
 Score: 257.34200000000004


## Apparatus
Selected Athletes: ['shane wiskus', 'khoi young', 'donnell whittenburg', 'brody malone']
 Score: 251.11566666666667

---


# brute-force search - Not working for using all apparatus 3times

### Apparatus
Selected Athletes:
['aidan cuy', 'asher hong', 'paul juda', 'yul moldauer']
 Score:
169.7672115079365

### IAA

Selected Athletes: ['vitaliy guimaraes', 'khoi young', 'asher hong', 'yul moldauer']  Score: 173.974

---


# MILP

### Apparatus
not returning 4 athletes

### IAA
not returning 4 athletes


In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from pulp import LpProblem, LpVariable, lpSum, LpMaximize


In [25]:
df = pd.read_csv('../../Data/cleandata22-23/usa_men22_23.csv')
df1 = pd.read_csv('../../Cleaning/USOPC/usa_men_apparatus.csv')
df2=pd.read_csv('update_usa_apparatus.csv')
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 

In [26]:
usa = iaa24[iaa24['Nation'] == 'USA']


In [28]:
usa.sort_values(by='AA', ascending=False)

,Rank,Athlete,FX,PH,SR,VT,PB,HB,AA,round,Nation,year
679,1.0,yul moldauer,14.400,13.957,14.114,14.250,15.121,13.500,85.342,final,USA,2023
680,2.0,fred richard,14.483,13.357,13.350,14.000,14.450,14.962,84.602,final,USA,2023
421,1.0,vitaliy guimaraes,14.250,14.350,13.600,14.450,13.900,13.400,83.950,final,USA,2022
422,2.0,khoi young,13.550,15.342,12.550,15.244,13.750,13.100,83.536,final,USA,2022
423,3.0,asher hong,14.029,12.949,14.566,16.680,14.355,10.450,83.029,final,USA,2022
...,...,...,...,...,...,...,...,...,...,...,...,...
716,38.0,ian skirkey,0.000,14.972,0.000,0.000,0.000,0.000,14.972,final,USA,2023
717,39.0,ignacio yockers,0.000,14.942,0.000,0.000,0.000,0.000,14.942,final,USA,2023
718,40.0,brandon dang,0.000,14.466,0.000,0.000,0.000,0.000,14.466,final,USA,2023
719,41.0,vahe petrosyan,0.000,0.000,0.000,13.350,0.000,0.000,13.350,final,USA,2023


In [4]:
df=usa.copy()

In [5]:

# Updated data without brute-force search
data = {
    'Algorithm': ['Tabu Search', 'Tabu Search', 'Hyper-Heuristics', 'Hyper-Heuristics', 'Variable Neighborhood Search (VNS)', 'Variable Neighborhood Search (VNS)'],
    'IAA/Apparatus': ['IAA', 'Apparatus', 'IAA', 'Apparatus', 'IAA', 'Apparatus'],
    'Selected Athletes': [
        ['khoi young', 'asher hong', 'fred richard', 'yul moldauer'],
        ['frederick nathaniel richard', 'paul juda', 'khoi young', 'brody malone'],
        ['joshua karnes', 'ian lasic-ellis', 'asher hong', 'yul moldauer'],
        ['khoi young', 'paul juda', 'brody malone', 'fuzzy benas'],
        ['fuzzy benas', 'yul moldauer', 'asher hong', 'fred richard'],
        ['shane wiskus', 'khoi young', 'donnell whittenburg', 'brody malone']
    ],
    'Total Score': [260.57900000000006, 253.19598690476192, 254.94, 250.58868690476191, 257.34200000000004, 251.11566666666667]
}

# Create DataFrame
result_df = pd.DataFrame(data)

# Add columns for each athlete
athletes = list(set(athlete for athletes_list in result_df['Selected Athletes'] for athlete in athletes_list))

for athlete in athletes:
    result_df[athlete] = result_df['Selected Athletes'].apply(lambda x: 1 if athlete in x else 0)

# Calculate the sum for each athlete column
athlete_sums = result_df.groupby('Algorithm')[athletes].sum()



# Approx 40% of Athletes have only 1 Entry, therefore I am taking each athletes best scores.

In [6]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df = df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_42681/1041535683.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa.fillna(0, inplace=True)


#  Variable Neighborhood Search (VNS):

## IAA

Selected Athletes: ['fuzzy benas', 'yul moldauer', 'asher hong', 'fred richard']
Total Score: 257.34200000000004


## Apparatus
Selected Athletes: ['shane wiskus', 'khoi young', 'donnell whittenburg', 'brody malone']
Total Score: 251.11566666666667


---

## IAA

In [7]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_42681/318956366.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa.fillna(0, inplace=True)


In [8]:
df.head()

,Athlete,FX,PH,SR,VT,PB,HB
0,alex diab,13.550,0.000,14.698,13.552,0.000,0.00
1,anthony koppie,11.900,4.650,12.550,13.950,11.750,11.40
2,asher hong,14.029,12.949,14.566,16.680,14.700,13.65
3,blake sun,0.000,13.500,0.000,0.000,15.044,0.00
4,brandon dang,0.000,14.466,0.000,0.000,0.000,0.00


In [13]:
# VNS Algorithm with Iteration through random_state values
def variable_neighborhood_search(df):
    apparatus_order = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']  

    def calculate_total_score(athletes_df):
        # Sum the top 3 scores for each apparatus
        total_score = athletes_df[['FX', 'PH', 'SR', 'VT', 'PB', 'HB']].apply(lambda x: x.nlargest(3).sum()).sum()
        return total_score

    best_athletes = None
    best_score = float('-inf')
    best_seed = None

    for seed in range(6000): 
        # Randomly select 4 athletes with the current seed
        selected_athletes = df.sample(4, random_state=seed)

        # Check if each apparatus is competed on 3 times
        if all(selected_athletes[apparatus].count() >= 3 for apparatus in apparatus_order):
            # Calculate the total score for the current solution
            current_score = calculate_total_score(selected_athletes)

            # Compare with the current best solution
            if current_score > best_score:
                best_athletes = selected_athletes['Athlete'].tolist()
                best_score = current_score
                best_seed = seed

    return best_athletes, best_score, best_seed

# Run the VNS algorithm with different random_state values
best_athletes, total_score, best_seed = variable_neighborhood_search(df)

print("Selected Athletes:", best_athletes)
print("Total Score:", total_score)
print("Best Seed:", best_seed)


Selected Athletes: ['asher hong', 'alex diab', 'anthony koppie', 'blake sun']
Total Score: 223.11800000000002
Best Seed: 0


## Apparatus
Selected Athletes:
['colt walker' 'paul juda' 'khoi young' 'frederick nathaniel richard']
Total Score:
170.307825



In [ ]:
df = df2.copy()

In [ ]:
df.sort_values(by='name', inplace=True)


In [ ]:
df.head()

In [ ]:
# VNS Algorithm with Iteration through random_state values
def variable_neighborhood_search(df):
    apparatus_order = ['fx', 'ph', 'sr', 'vt', 'pb', 'hb']  # Define the order of apparatus

    def calculate_total_score(athletes_df):
        # Sum the top 3 scores for each apparatus
        total_score = athletes_df[['fx', 'ph', 'sr', 'vt', 'pb', 'hb']].apply(lambda x: x.nlargest(3).sum()).sum()
        return total_score

    best_athletes = None
    best_score = float('-inf')
    best_seed = None

    for seed in range(7000):  # Try different random_state values
        # Randomly select 4 athletes with the current seed
        selected_athletes = df.sample(4, random_state=seed)

        # Check if each apparatus is competed on 3 times
        if all(selected_athletes[apparatus].count() >= 3 for apparatus in apparatus_order):
            # Calculate the total score for the current solution
            current_score = calculate_total_score(selected_athletes)

            # Compare with the current best solution
            if current_score > best_score:
                best_athletes = selected_athletes['name'].tolist()
                best_score = current_score
                best_seed = seed

    return best_athletes, best_score, best_seed

# Run the VNS algorithm with different random_state values
best_athletes, total_score, best_seed = variable_neighborhood_search(df)

# Display the best solution and the seed that produced it
print("Selected Athletes:", best_athletes)
print("Total Score:", total_score)
print("Best Seed:", best_seed)


In [ ]:
# Selected Athletes: ['donnell whittenburg', 'ian lasic-ellis', 'paul juda', 'asher hong']
# Total Score: 249.27948650793653
# Best Seed: 749


# Selected Athletes: ['shane wiskus', 'khoi young', 'donnell whittenburg', 'brody malone']
# Total Score: 251.11566666666667
# Best Seed: 2981


# Hyper-Heuristics:

## IAA
Best Selection: ['joshua karnes' 'ian lasic-ellis' 'asher hong' 'yul moldauer']
Best Score: 254.94


## Apparatus
Best Selection: ['khoi young' 'paul juda' 'brody malone' 'fuzzy benas']
Best Score: 250.58868690476191

---

## IAA

In [ ]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

In [ ]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['Athlete'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


## Apparatus
Best Selection: ['khoi young' 'paul juda' 'brody malone' 'fuzzy benas']
Best Score: 250.58868690476191

In [ ]:
df = df2.copy()

In [ ]:
df.info()

In [ ]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame
# df = ...

# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['fx', 'ph', 'sr', 'vt', 'pb', 'hb']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['name'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['name'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


# Tabu Search

## IAA

Selected Athletes:
['khoi young' 'asher hong' 'fred richard' 'yul moldauer']
Total Score:
260.57900000000006

## Apparatus
Selected Athletes:
['frederick nathaniel richard' 'paul juda' 'khoi young' 'brody malone']
Total Score:
253.19598690476192

---

## IAA

In [ ]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])
df.head()

In [ ]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['Athlete'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop(['Athlete'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['Athlete'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['Athlete']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score



# Set parameters
num_athletes_to_select = 4
tabu_size = 10
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

## Apparatus
Selected Athletes:
['frederick nathaniel richard' 'paul juda' 'khoi young' 'brody malone']
Total Score:
253.19598690476192

In [ ]:
df=df2.copy()

In [ ]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['name'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop([ 'name'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['name'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['name']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score


# Set parameters
num_athletes_to_select = 4
tabu_size = 300
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

# MILP solver 


## IAA

In [ ]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

df.head()

In [ ]:
import pulp

def milp_selection(df):
    # Define the apparatus names
    apparatus_order = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

    # Create a binary variable for each gymnast
    selected = pulp.LpVariable.dicts("Selected", df.index, cat=pulp.LpBinary)

    # Create the LP problem
    prob = pulp.LpProblem("Gymnast_Selection", pulp.LpMaximize)

    # Objective function: maximize the total score
    prob += pulp.lpSum(selected[i] * df.at[i, apparatus] for i in df.index for apparatus in apparatus_order)

    # Constraints
    # Select exactly 4 gymnasts
    prob += pulp.lpSum(selected[i] for i in df.index) == 4

    # Each gymnast should compete on each apparatus at least once
    for i in df.index:
        prob += pulp.lpSum(selected[i] for apparatus in apparatus_order) >= 1

    # Each gymnast should compete on each apparatus at most three times
    for i in df.index:
        prob += pulp.lpSum(selected[i] for apparatus in apparatus_order) <= 3

    # Solve the problem
    prob.solve()

    # Extract the selected gymnasts
    selected_gymnasts = [i for i in df.index if pulp.value(selected[i]) == 1]

    # Extract the total score
    total_score = pulp.value(prob.objective)

    return selected_gymnasts, total_score

# Create the DataFrame
import pandas as pd

# Run the MILP model
selected_gymnasts, total_score = milp_selection(df)

# Display the results
print("Selected Gymnasts:", selected_gymnasts)
print("Total Score:", total_score)


## Apparatus

In [ ]:
df=df2.copy()

In [ ]:
df.head()

In [ ]:
# import pulp

# def milp_selection(df):
#     # Define the apparatus names
#     apparatus_order = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

#     # Create a binary variable for each gymnast
#     selected = pulp.LpVariable.dicts("Selected", df.index, cat=pulp.LpBinary)

#     # Create the LP problem
#     prob = pulp.LpProblem("Gymnast_Selection", pulp.LpMaximize)

#     # Objective function: maximize the total score
#     prob += pulp.lpSum(selected[i] * df.at[i, apparatus] for i in df.index for apparatus in apparatus_order)

#     # Constraints
#     # Select exactly 4 gymnasts
#     prob += pulp.lpSum(selected[i] for i in df.index) == 4

#     # Each gymnast should compete on each apparatus at least once
#     for i in df.index:
#         prob += pulp.lpSum(selected[i] for apparatus in apparatus_order) >= 1

#     # Each gymnast should compete on each apparatus at most three times
#     for i in df.index:
#         prob += pulp.lpSum(selected[i] for apparatus in apparatus_order) <= 3

#     # Solve the problem
#     prob.solve()

#     # Extract the selected gymnasts
#     selected_gymnasts = [i for i in df.index if pulp.value(selected[i]) == 1]

#     # Extract the total score
#     total_score = pulp.value(prob.objective)

#     return selected_gymnasts, total_score

# # Create the DataFrame
# import pandas as pd

# # Run the MILP model
# selected_gymnasts, total_score = milp_selection(df)

# # Display the results
# print("Selected Gymnasts:", selected_gymnasts)
# print("Total Score:", total_score)


# brute-force search

## Apparatus
Selected Athletes:
['aidan cuy', 'asher hong', 'paul juda', 'yul moldauer']
Maximum Score:
169.7672115079365

## IAA

Selected Athletes: ['vitaliy guimaraes', 'khoi young', 'asher hong', 'yul moldauer'] Maximum Score: 173.974


---

## Apparatus


In [ ]:
df=df2.copy()

In [ ]:
df.head()

Over 3 hours to run
100%|█████████████████████████████████| 424270/424270 [3:13:04<00:00, 36.62it/s]

Selected Athletes:
['aidan cuy', 'asher hong', 'paul juda', 'yul moldauer']
Maximum Score:
169.7672115079365


In [ ]:
df.info()

## IAA

Selected Athletes:
['vitaliy guimaraes', 'khoi young', 'asher hong', 'yul moldauer']
Maximum Score:
173.974

In [ ]:
import pandas as pd
import itertools
from tqdm import tqdm

# Assuming you have a DataFrame 'df' with columns 'name', 'fx', 'hb', 'pb', 'ph', 'sr', 'vt'
# Replace this with your actual data
# For example:
# df = pd.DataFrame({
#     'name': ['aidan cuy', 'alex diab', 'asher hong'],
#     'fx': [0.0, 13.225, 13.845625],
#     'hb': [0.0, 0.0, 12.072875],
#     'pb': [12.95, 0.0, 14.579444],
#     'ph': [0.0, 0.0, 13.124],
#     'sr': [12.3, 14.144167, 13.999778],
#     'vt': [13.8, 0.0, 14.393643]
# })

# Generate all possible combinations of athletes and apparatuses
athletes = df['name'].unique()
apparatuses = df.columns[1:]  # Exclude the 'name' column
combinations = list(itertools.product(athletes, apparatuses))

# Initialize variables to store the best combination and maximum total score
best_combination = None
max_total_score = 0

# Iterate through all combinations with tqdm for a progress bar
for combo in tqdm(itertools.permutations(combinations, 4), total=len(list(itertools.permutations(combinations, 4)))):
    # Create a DataFrame with the selected combination
    selected_df = pd.DataFrame(list(combo), columns=['name', 'Apparatus'])
    
    # Ensure each athlete competes on each apparatus at most once
    if selected_df.duplicated(subset='name').any() or selected_df.duplicated(subset='Apparatus').any():
        continue
    
    # Ensure each apparatus is competed on 3 times
    if selected_df['Apparatus'].value_counts().min() < 3:
        continue
    
    # Calculate the total score for the current combination
    total_score = 0
    for _, row in selected_df.iterrows():
        total_score += df.loc[df['name'] == row['name'], row['Apparatus']].max()
    
    # Update the best combination if the total score is higher
    if total_score > max_total_score:
        max_total_score = total_score
        best_combination = selected_df

# Print the best combination and the maximum total score
print("\nBest Combination:")
print(best_combination)
print("Maximum Total Score:", max_total_score)


In [ ]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

df.head()

In [ ]:
df.info()

In [ ]:
# import pandas as pd
# import itertools
# from tqdm import tqdm

# num_athletes = 4
# apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# # Generate combinations of athletes
# athlete_combinations = list(itertools.combinations(df['Athlete'], num_athletes))

# # Initialize variables to store  maximum score and  selected athletes
# max_score = 0
# selected_athletes = []

# progress_bar = tqdm(total=len(athlete_combinations))

# # Iterate through  combination of athletes
# for combination in athlete_combinations:
#     # Generate combinations of apparatuses for  current athlete combination
#     apparatus_combinations = list(itertools.combinations(apparatuses, num_athletes))

#     # Iterate through combination of apparatuses
#     for apparatus_combination in apparatus_combinations:
#         # Calculate sum of top three scores for current apparatus combination
#         score = df[df['Athlete'].isin(combination)][list(apparatus_combination)].sum(axis=1).nlargest(3).sum()

#         # Update maximum score and selected athletes if a higher score is found
#         if score > max_score:
#             max_score = score
#             selected_athletes = list(combination)

#     progress_bar.update(1)

# progress_bar.close()

# print('Selected Athletes:')
# print(selected_athletes)
# print('Maximum Score:')
# print(max_score)

---

# Simulated Annealing
- Simulated annealing involves randomness in accepting worse solutions to explore the search space effectively. After adding Seed the results are terrible


Selected Athletes: ['fuzzy benas', 'yul moldauer', 'asher hong', 'fred richard']
Maximum Score: 174.61700000000002



In [ ]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

df.head()

In [ ]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    total_score = df[df['Athlete'].isin(selected_athletes)].iloc[:, 2:8].values
    total_score.sort(axis=1, kind='quicksort')
    return total_score[:, -3:].sum()

def simulated_annealing(df, num_athletes=4, iterations=1000, initial_temperature=1.0, cooling_rate=0.95):
    np.random.seed(88)  # Set the seed for reproducibility

    current_solution = np.random.choice(df['Athlete'], size=num_athletes, replace=False).tolist()
    current_score = calculate_score(current_solution, df)

    best_solution = current_solution.copy()
    best_score = current_score

    temperature = initial_temperature

    for iteration in range(iterations):
        neighbor_solution = current_solution.copy()
        idx1, idx2 = np.random.choice(num_athletes, size=2, replace=False)
        neighbor_solution[idx1], neighbor_solution[idx2] = neighbor_solution[idx2], neighbor_solution[idx1]

        neighbor_score = calculate_score(neighbor_solution, df)

        if neighbor_score > current_score or np.random.rand() < np.exp((neighbor_score - current_score) / temperature):
            current_solution = neighbor_solution
            current_score = neighbor_score

        if current_score > best_score:
            best_solution = current_solution.copy()
            best_score = current_score

        temperature *= cooling_rate

    return best_solution, best_score


# Run simulated annealing
best_athletes, best_score = simulated_annealing(df)

# Print the results
print('Selected Athletes:', best_athletes)
print('Maximum Score:', best_score)


# Harmony Search algorithm = results for 6 and not 4 athletes

              Athlete     FX      PH     SR     VT      PB     HB
715    patrick hoopes   0.00  14.992   0.00   0.00   0.000   0.00
708   curran phillips   0.00   0.000   0.00   0.00  15.997  13.70
427   ian lasic-ellis  13.65  13.500  13.75  13.55  13.300  13.30
711   spencer goodell  13.45   0.000   0.00   0.00  13.750   0.00
442      noah newfeld  12.20  13.050  12.10  13.50  13.200  12.15
444  garrett braunton  12.85  12.900  13.05  12.20  13.150  11.65

In [ ]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

df.head()

In [ ]:


import pandas as pd
import numpy as np

def fitness(solution, df):
    selected_athletes = df.iloc[solution]
    
    # Check if each apparatus is competed exactly three times
    apparatus_counts = selected_athletes.groupby('Athlete').size()
    if not all(count == 3 for count in apparatus_counts):
        return np.inf
    
    # Check if the selected athletes are unique for each apparatus
    if len(set(solution)) < len(solution):
        return np.inf
    
    scores = selected_athletes[['FX', 'PH', 'SR', 'VT', 'PB', 'HB']].values
    top3_sum = np.sum(np.partition(scores, -3)[:, -3:], axis=1)
    
    return -np.sum(top3_sum)  # Negative because Harmony Search seeks to minimize the objective function

def harmony_search(df, num_iterations, harmony_memory_size):
    num_athletes = len(df)
    num_apparatus = 6  # Assuming 6 apparatus (FX, PH, SR, VT, PB, HB)
    hm_size = harmony_memory_size

    # Initialize harmony memory with random solutions
    harmony_memory = np.random.randint(0, num_athletes, size=(hm_size, num_apparatus))

    for iteration in range(num_iterations):
        # Create new harmony solutions
        new_solution = np.random.randint(0, num_athletes, size=num_apparatus)

        # Update the harmony memory with the new solution
        worst_index = np.argmin([fitness(harmony, df) for harmony in harmony_memory])
        
        # Check if the new solution is valid before updating the memory
        if fitness(new_solution, df) < np.inf:
            harmony_memory[worst_index] = new_solution

    # Find the best solution in the harmony memory
    best_solution = harmony_memory[np.argmin([fitness(harmony, df) for harmony in harmony_memory])]

    return df.iloc[best_solution]

# Assuming 'your_dataframe.csv' is the path to your CSV file
# df = pd.read_csv('your_dataframe.csv')

# Set the number of iterations and harmony memory size
num_iterations = 100
harmony_memory_size = 10

# Run the Harmony Search algorithm
selected_athletes = harmony_search(df, num_iterations, harmony_memory_size)

# Print the selected athletes
print(selected_athletes[['Athlete', 'FX', 'PH', 'SR', 'VT', 'PB', 'HB']])


# Stochastic control

## IAA

In [ ]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

In [ ]:
df.columns

In [ ]:
import pandas as pd
from itertools import combinations
from tqdm import tqdm



# Get all possible combinations of 4 gymnasts out of 5
gymnast_combinations = list(combinations(df['Athlete'], 4))

# Initialize variables to keep track of the best result
best_result = None
best_sum = float('-inf')

# Iterate over all combinations of gymnasts with tqdm progress bar
for combination in tqdm(gymnast_combinations, desc="Combinations"):
    # Select the rows corresponding to the current combination of gymnasts
    selected_gymnasts = df[df['Athlete'].isin(combination)]
    
    # Ensure each apparatus is competed on 3 times by each selected gymnast
    selected_gymnasts = selected_gymnasts.groupby('Athlete').apply(lambda x: x.sample(n=3, replace=True, random_state=42)).reset_index(drop=True)
    
    # Calculate the sum of the top 3 scores for each apparatus
    selected_gymnasts['Top3Sum'] = selected_gymnasts[['FX', 'PH', 'SR', 'VT', 'PB', 'HB']].apply(lambda x: sum(sorted(x, reverse=True)[:3]), axis=1)
    
    # Get the largest sum for each apparatus
    current_sum = selected_gymnasts.groupby('Athlete').agg({'Top3Sum': 'max'}).reset_index()['Top3Sum'].sum()
    
    # Update the best result if the current combination is better
    if current_sum > best_sum:
        best_sum = current_sum
        best_result = selected_gymnasts[['Athlete', 'Top3Sum']].copy()

# Print the best result
print(best_result)


In [ ]:
41.80+71.95+77.55+76.25

## Apparatus

In [ ]:
df=df2.copy()

In [ ]:
df

In [ ]:
import pandas as pd
from itertools import combinations
from tqdm import tqdm



# Get all possible combinations of 4 gymnasts out of 5
gymnast_combinations = list(combinations(df['name'], 4))

# Initialize variables to keep track of the best result
best_result = None
best_sum = float('-inf')

# Iterate over all combinations of gymnasts with tqdm progress bar
for combination in tqdm(gymnast_combinations, desc="Combinations"):
    # Select the rows corresponding to the current combination of gymnasts
    selected_gymnasts = df[df['name'].isin(combination)]
    
    # Ensure each apparatus is competed on 3 times by each selected gymnast
    selected_gymnasts = selected_gymnasts.groupby('name').apply(lambda x: x.sample(n=3, replace=True, random_state=42)).reset_index(drop=True)
    
    # Calculate the sum of the top 3 scores for each apparatus
    selected_gymnasts['Top3Sum'] = selected_gymnasts[['fx', 'hb', 'pb', 'ph', 'sr', 'vt']].apply(lambda x: sum(sorted(x, reverse=True)[:3]), axis=1)
    
    # Get the largest sum for each apparatus
    current_sum = selected_gymnasts.groupby('name').agg({'Top3Sum': 'max'}).reset_index()['Top3Sum'].sum()
    
    # Update the best result if the current combination is better
    if current_sum > best_sum:
        best_sum = current_sum
        best_result = selected_gymnasts[['name', 'Top3Sum']].copy()

# Print the best result
print(best_result)


In [ ]:
78.149667+81.309627+84.374795+39.050000

In [ ]:
6*14.00